In [8]:
import sys
sys.path.append('../src')
sys.path.append('../scripts')

import warnings
warnings.filterwarnings('ignore')

In [9]:
from jobDescriptionScraper import LinkedInDescriptionScraper
from jobListingScraper import LinkedInJobListingScraper

In [10]:
from resumeRestructre import save_restructured_resume

In [11]:
# Restructuring the input resume
restructured_resume_path, restructured_resume_json= save_restructured_resume('inputs\sample_resume_input_1.pdf', 1)

In [30]:
%%time
# Job Listing Scraping
job_listing_scraper = LinkedInJobListingScraper()
jobs = job_listing_scraper.scrape_linkedin_jobs("Machine Learning Engineer", "India", 5, "../linkedincookies.pkl")

Scraping page starting at 0
CPU times: total: 141 ms
Wall time: 31.1 s


In [31]:
%%time
# Job Description Scraping
job_description_scraper = LinkedInDescriptionScraper()
jobs['Job Descriptions'] = jobs['Job ID'].apply(lambda x : job_description_scraper.getDescription(job_id=x))

CPU times: total: 15.6 ms
Wall time: 13.1 s


In [32]:
from pdfLoader import loadPdfContent
restructured_resume_path = "..\inputs\sample_resume_input_1.pdf"
old_resume_content = loadPdfContent(restructured_resume_path)

In [33]:
# Old Resume Scoring
from resumeRelevancyScore import RelevanceScorer
resume_relevence_scorer = RelevanceScorer()

jobs['Old Resume Score'] = jobs['Job Descriptions'].apply(lambda x : resume_relevence_scorer.calculate_relevance_score(old_resume_content, x))

In [34]:
jobs

,Job Title,Job Post Url,Company Name,Location,Job ID,Job Descriptions,Old Resume Score
0,Generative AI Engineer,https://www.linkedin.com/jobs/view/4009701610/...,Deloitte,"Bengaluru, Karnataka, India (Hybrid)",4009701610,"Hi, We are looking for Candidates who are havi...",66.19
1,Data and Applied Scientist II,https://www.linkedin.com/jobs/view/3977751872/...,Microsoft,"Kerala, India (Hybrid)",3977751872,Microsoft’s Cloud business is experiencing exp...,66.57
2,Data Scientist,https://www.linkedin.com/jobs/view/4027402513/...,GeorgiaTEK Systems Inc.,India (Remote),4027402513,Data Scientist\nRemote\nFulltime\n\nCandidate ...,66.36
3,Data Scientist,https://www.linkedin.com/jobs/view/4027415095/...,GeorgiaTEK Systems Inc.,India (Remote),4027415095,Job Title: Data Scientist\nExperience: 6+ year...,65.85
4,Senior Machine Learning Engineer,https://www.linkedin.com/jobs/view/4011065293/...,BayOne Solutions,Greater Bengaluru Area (Hybrid),4011065293,Description\nMust Have Skills\nProven experien...,69.51


In [35]:
from reGenerate import regen_resume
import pandas as pd

def apply_regen_resume(row):
    regen_json, output_filepath = regen_resume(row['Job Descriptions'], restructured_resume_json, row['Job ID'])
    return pd.Series({'ReGen JSON': regen_json, 'ReGen FilePath': output_filepath})

jobs[['ReGen JSON', 'ReGen FilePath']] = jobs.apply(apply_regen_resume, axis=1)

In [59]:
import importlib
from resumeRelevancyScore import RelevanceScorer

# Your code using RelevanceScorer...

# When you want to reload the module:
importlib.reload(importlib.import_module('resumeRelevancyScore'))
from resumeRelevancyScore import RelevanceScorer

In [60]:
from resumeRelevancyScore import RelevanceScorer
resume_relevence_scorer = RelevanceScorer()

# Calculating the relavancy score of new resume
jobs['New Resume Score'] = jobs[['Job Descriptions', 'Job ID']].apply(lambda x : resume_relevence_scorer.calculate_relevance_score_new(x[1], x[0]), axis=1)

In [57]:
jobs

,Job Title,Job Post Url,Company Name,Location,Job ID,Job Descriptions,Old Resume Score,ReGen JSON,ReGen FilePath,New Resume Score
0,Generative AI Engineer,https://www.linkedin.com/jobs/view/4009701610/...,Deloitte,"Bengaluru, Karnataka, India (Hybrid)",4009701610,"Hi, We are looking for Candidates who are havi...",66.19,"{'name': 'Muhammed Büyükkınacı', 'contact': {'...",../generatedResumes\4009701610.pdf,80.35
1,Data and Applied Scientist II,https://www.linkedin.com/jobs/view/3977751872/...,Microsoft,"Kerala, India (Hybrid)",3977751872,Microsoft’s Cloud business is experiencing exp...,66.57,"{'name': 'Muhammed Büyükkınacı', 'contact': {'...",../generatedResumes\3977751872.pdf,67.46
2,Data Scientist,https://www.linkedin.com/jobs/view/4027402513/...,GeorgiaTEK Systems Inc.,India (Remote),4027402513,Data Scientist\nRemote\nFulltime\n\nCandidate ...,66.36,"{'name': 'Muhammed Büyükkınacı', 'contact': {'...",../generatedResumes\4027402513.pdf,71.93
3,Data Scientist,https://www.linkedin.com/jobs/view/4027415095/...,GeorgiaTEK Systems Inc.,India (Remote),4027415095,Job Title: Data Scientist\nExperience: 6+ year...,65.85,"{'name': 'Muhammed Büyükkınacı', 'contact': {'...",../generatedResumes\4027415095.pdf,80.94
4,Senior Machine Learning Engineer,https://www.linkedin.com/jobs/view/4011065293/...,BayOne Solutions,Greater Bengaluru Area (Hybrid),4011065293,Description\nMust Have Skills\nProven experien...,69.51,"{'name': 'Muhammed Büyükkınacı', 'contact': {'...",../generatedResumes\4011065293.pdf,86.95
